In [4]:
import xarray as xr
import numpy as np
# plotting
import matplotlib.pyplot as plt
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio 
## import dask dependencies
import dask
from dask import delayed

In [10]:
# functions I will need 
def apogee2temp(dat,tower):
# hard-coded sensor-specific calibrations
    Vref = 2.5
    ID = dat[f"IDir_{tower}"]
    sns = [136, 137, 138, 139, 140]
    im = [ sns.index(x) if x in sns else None for x in ID ][0]
# unclear if we want these, or scaled up versions
    mC0 = [57508.575,56653.007,58756.588,58605.7861,58756.588][im] * 1e5
    mC1 = [289.12189,280.03380,287.12487,285.00285,287.12487][im] * 1e5
    mC2 = [2.16807,2.11478,2.11822,2.08932,2.11822][im] * 1e5
    bC0 = [-168.3687,-319.9362,-214.5312,-329.6453,-214.5312][im]* 1e5
    bC1 = [-0.22672,-1.23812,-0.59308,-1.24657,-0.59308][im]* 1e5
    bC2 = [0.08927,0.08612,0.10936,0.09234,0.10936][im]* 1e5
# read data
    Vtherm = dat[f"Vtherm_{tower}"]
    Vpile = dat[f"Vpile_{tower}"]*1000
# calculation of detector temperature from Steinhart-Hart
    Rt = 24900.0/((Vref/Vtherm) - 1)
    Ac = 1.129241e-3
    Bc = 2.341077e-4
    Cc = 8.775468e-8
    TDk = 1/(Ac + Bc*np.log(Rt) + Cc*(np.log(Rt)**3))
    TDc = TDk - 273.15
# finally, calculation of "target" temperature including thermopile measurement
    m = mC2*TDc**2 + mC1*TDc + mC0
    b = bC2*TDc**2 + bC1*TDc + bC0
    TTc = (TDk**4 + m*Vpile + b)**0.25 - 273.15
    # sufs = suffixes(TTc,leadch='') # get suffixes
    # dimnames(TTc)[[2]] = paste0("Tsfc.Ap.",sufs)
    return TTc


In [15]:
# check to see if sos_ds_5min_storage_with_tsurf.nc exists
if os.path.exists('../../01_data/processed_data/sos_ds_5min_storage_with_tsurf.nc'):
    print('sos_ds_5min_storage_with_tsurf.nc exists')
    # load the dataset
    sos_5min_ds = xr.open_dataset('../../01_data/processed_data/sos_ds_5min_storage_with_tsurf.nc')
else:
    print('Calculating Surface Temperatures...')
    # surface temperature
    sos_5min_ds = sos_5min_ds.assign(Tsurfapogee_d=apogee2temp(sos_5min_ds,'d'))
    print('1/4 towers complete...')
    sos_5min_ds = sos_5min_ds.assign(Tsurfapogee_c=apogee2temp(sos_5min_ds,'c'))
    print('2/4 towers complete...')
    sos_5min_ds = sos_5min_ds.assign(Tsurfapogee_uw=apogee2temp(sos_5min_ds,'uw'))
    print('3/4 towers complete...')
    sos_5min_ds = sos_5min_ds.assign(Tsurfapogee_ue=apogee2temp(sos_5min_ds,'ue'))
    print('4/4 towers complete... Saving dataset...')
    # save the updated dataset
    sos_5min_ds.to_netcdf('../../01_data/processed_data/sos_ds_5min_storage_with_tsurf.nc')
    print('Dataset saved!')

sos_ds_5min_storage_with_tsurf.nc exists


In [30]:
df = sos_5min_ds[['Tsurfapogee_d', 'Tsurfapogee_c','Tsurfapogee_uw','Tsurfapogee_ue']].sel(time=slice('2023-05-01','2023-05-03')).to_dataframe().resample('1H').mean()

In [31]:
fig = go.Figure()
for column in df.columns:
    fig.add_trace(go.Scatter(x=df.index, y=df[column], mode='lines', name=column))
fig.update_layout(title='Surface Temperatures', xaxis_title='Time', yaxis_title='Temperature (C)')
fig.show()